In [4]:
import os
import pandas as pd
import time
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.fundamentaldata import FundamentalData
from fredapi import Fred

try:
    import yfinance as yf
except ImportError:
    print("⚠️ yfinance not installed. Install with: pip install yfinance")
    yf = None

av_key = os.getenv('ALPHA_VANTAGE_KEY')
fred_key = os.getenv('FRED_API_KEY')

ts = TimeSeries(key=av_key, output_format='pandas')
fd = FundamentalData(key=av_key, output_format='pandas')
fred = Fred(api_key=fred_key)

# Cache für Daten
cache = {}

def fetch_market_state(symbol='SMX'):
    """Fetch market state and pricing data for a given symbol"""
    global cache
    
    print(f"📡 Fetching data for {symbol}...")
    
    # Check if data is already cached
    if symbol in cache:
        print("✓ Using cached data...")
        current_price, pe_ratio = cache[symbol]
    else:
        try:
            # Try Alpha Vantage first
            print("  Trying Alpha Vantage API...")
            price_df, _ = ts.get_daily(symbol=symbol, outputsize='compact')
            current_price = price_df['4. close'].iloc[-1]
            
            # Add delay to avoid rate limits
            time.sleep(0.5)
            
            # Get company overview
            overview, _ = fd.get_company_overview(symbol=symbol)
            pe_ratio = overview.get('PERatio', 'N/A')
            
            print("  ✓ Alpha Vantage API successful")
            
        except Exception as e:
            # Fallback to Yahoo Finance
            print(f"  ⚠️ Alpha Vantage failed: {str(e)}")
            print("  Falling back to Yahoo Finance...")
            
            if yf is None:
                print("  Error: yfinance not available")
                return
            
            try:
                ticker = yf.Ticker(symbol)
                hist = ticker.history(period='1d')
                current_price = hist['Close'].iloc[-1]
                
                # Get PE ratio from yfinance
                pe_ratio = ticker.info.get('trailingPE', 'N/A')
                
                print("  ✓ Yahoo Finance successful")
            except Exception as e2:
                print(f"  Error fetching from Yahoo Finance: {str(e2)}")
                return
        
        # Cache the data
        cache[symbol] = (current_price, pe_ratio)

    # Get latest Federal Interest Rates
    try:
        interest_rate = fred.get_series('FEDFUNDS').iloc[-1]
    except:
        interest_rate = 'N/A'

    # Final Summary
    print(f"\n✅ Sucess! Current State:")
    print(f"    Price: $ {current_price:.2f}")
    print(f" P/E Ratio: {pe_ratio}")
    print(f"     US Interest Rate: {interest_rate}%\n")

fetch_market_state('SMX')

⚠️ yfinance not installed. Install with: pip install yfinance
📡 Fetching data for SMX...
  Trying Alpha Vantage API...
  ⚠️ Alpha Vantage failed: We have detected your API key as HS73WTF9MRQ4ET56 and our standard API rate limit is 25 requests per day. Please subscribe to any of the premium plans at https://www.alphavantage.co/premium/ to instantly remove all daily rate limits.
  Falling back to Yahoo Finance...
  Error: yfinance not available
